In [45]:
import numpy as np
import gym

import torch
import json
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim, distributions
import cv2

import random                 # Handling random number generation
import time                   # Handling time calculation
from skimage import transform # Help us to preprocess the frames

from collections import deque # Ordered collection with ends
import matplotlib.pyplot as plt  # Display graphs

import warnings                  # This ignore all the warning messages that are normally printed during the training because of skiimage
warnings.filterwarnings('ignore')
import os


### MODEL HYPERPARAMETERS
learning_rate =  0.001   # Alpha (aka learning rate)
total_episodes = 100000

### TRAINING HYPERPARAMETERS
total_episodes = 100000         # Total episodes for training

# clip_norm = 0.1

model_folder = f"./models/reinforce/lr/LR{learning_rate}"
os.makedirs(model_folder)
log_path = f"./logs/reinforce/lr/LR{learning_rate}.txt"

env = gym.make('CartPole-v1')

class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(4, 32)
        self.layer2 = nn.Linear(32, 2)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, state):
        x = F.relu(self.layer1(state))
        x = self.layer2(x)
        return x
    
model = Net()
opt = optim.Adam(params=model.parameters(), lr=learning_rate)




In [46]:


class Net(nn.Module):

    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(4, 32)
        self.layer2 = nn.Linear(32, 2)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, state):
        x = F.relu(self.layer1(state))
        x = self.layer2(x)
        return x
    
model = Net()
opt = optim.Adam(params=model.parameters(), lr=learning_rate)

out_f = open(log_path, 'w')

total_steps = 0

for episode in range(total_episodes):

    action_log_probs = []

    episode_entropy = 0.0
    episode_steps = 0
    
    episode_reward = 0

    episode_argmax_action_taken = 0

    state, _ = env.reset()

    for time_steps in range(10000000000):

        # choose state from model
        tensor_state = torch.from_numpy(state).float().unsqueeze(0)
        action_logits = model(tensor_state)
        action_probs = F.softmax(action_logits, dim=1)

        # print(episode_entropy)
        # asdf
        m = distributions.Categorical(action_probs)
        action = m.sample()
        log_prob = m.log_prob(action)

        
        action_value = action.item()
        action_log_probs.append(log_prob)


        # Makes an action (here random one) and returns a reward.
        next_state, reward, done, _, _ = env.step(action_value)
        episode_steps += 1
        total_steps += 1

        episode_reward += reward

        state = next_state

        if done:
            per_timestep_losses = [- log_prob * episode_reward for log_prob in action_log_probs]
            # per_timestep_losses_t = torch.stack(per_timestep_losses)
            policy_loss = torch.cat(per_timestep_losses).sum()

            opt.zero_grad()
            policy_loss.backward()
            # torch.nn.utils.clip_grad_norm_(model.parameters(), clip_norm)
            opt.step()

            print(f"Episode {episode}     |      Reward: {episode_reward}     |      Total timesteps: {total_steps}")

            out_f.write(json.dumps({
                'episode': episode,
                'reward': episode_reward,
                'total_steps': total_steps
                }) + '\n')
            
            out_f.flush()

            if episode % 100 == 0:
                torch.save(model, f'{model_folder}/E_{episode}.pt')
                print(f'====== Model saved ======')
            break

    if total_steps > 200000:
        break

env.close()



Episode 0     |      Reward: 28.0     |      Total timesteps: 28
====== Model saved ======
Episode 1     |      Reward: 20.0     |      Total timesteps: 48
Episode 2     |      Reward: 11.0     |      Total timesteps: 59
Episode 3     |      Reward: 11.0     |      Total timesteps: 70
Episode 4     |      Reward: 24.0     |      Total timesteps: 94
Episode 5     |      Reward: 13.0     |      Total timesteps: 107
Episode 6     |      Reward: 18.0     |      Total timesteps: 125
Episode 7     |      Reward: 10.0     |      Total timesteps: 135
Episode 8     |      Reward: 14.0     |      Total timesteps: 149
Episode 9     |      Reward: 16.0     |      Total timesteps: 165
Episode 10     |      Reward: 10.0     |      Total timesteps: 175
Episode 11     |      Reward: 15.0     |      Total timesteps: 190
Episode 12     |      Reward: 12.0     |      Total timesteps: 202
Episode 13     |      Reward: 13.0     |      Total timesteps: 215
Episode 14     |      Reward: 22.0     |      Total